In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Professor names
professor_names = ['김유섭', '김은주', '이정근', '양은샘', '신미영', '김선정']

### “모르겠다”, “수업을 안들어서 모른다”, “기억 안남” 등의 응답을 NaN 처리하고 결측값 제거하기

1. **문장 유사도 기반 NaN 처리**  
   - `SentenceTransformer`를 이용해 기준 문장(“모르겠다”, “들어본 적 없음”, “기억 안난다” 등)을 먼저 임베딩하여 저장  
   - 각 리뷰 문장에 대해 동일한 임베딩 모델로 벡터화하고, 기준 임베딩들과의 코사인 유사도를 계산  
   - 유사도가 임계치(예: 0.85) 이상인 경우 `np.nan`으로 반환하여 “모름” 응답을 결측값으로 표시  

2. **결측값 제거**  
   - Pandas의 `dropna()` 또는 `DataFrame.dropna(axis=0, how='any')` 메서드를 사용해 NaN이 된 행(응답)들을 제거  
   - 이렇게 하면 실제 수업을 들은 경험에 기반한 리뷰만 남겨, 이후 분석의 왜곡을 막습니다  

In [ ]:
df = pd.read_excel('professor_review.xlsx')

In [ ]:
df.head()

In [ ]:
df = df.melt(
    id_vars=['응답일시', '참여자'],
    value_vars=df.columns[2:],
    var_name='professor',
    value_name='review'
)

# Extract the professor name without the title
df['professor'] = df['professor'].str.replace(' 교수님', '')

df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
# 결측값 처리 전
df['professor'].value_counts()

### Generate embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# 1) 임베딩 모델 로드 (가볍고 빠른 모델 추천)
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 2) 기준 문장 리스트 (수업 안 들음/모름 의미)
reference_texts = [
    '수업을 들은 적이 없다',
    '수강한 적 없음',
    '기억이 안 난다',
    '모름',
    '머름',
    'ㅡ'
]

# 기준 문장 임베딩
ref_embeddings = model.encode(reference_texts, convert_to_tensor=True)

def to_nan_if_similar(text, threshold=0.55):
    emb = model.encode(text, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(emb, ref_embeddings)
    max_score = cosine_scores.max().item()
    if max_score > threshold:
        print(f'NaN 처리됨: "{text}", 유사도: {max_score}')
        return np.nan
    return text

# professor_cols는 교수님별 컬럼 리스트
df['review'] = df['review'].apply(to_nan_if_similar)

In [ ]:
df.dropna(inplace=True)

In [ ]:
# 결측값 처리 후
df['professor'].value_counts()

### Save

In [ ]:
df.to_csv('survey_reviews.csv', index=False, encoding='utf-8-sig')